In [1]:
from connection import connect_
import sys
import pandas as pd
import json
from SDK import *

In [ ]:
#global variables : 
def get_db_detail():
    #username = os.environ.get('DB_USERNAME')
    #pwd = os.environ.get('DB_PASSWORD')
    #host = os.environ.get('DB_URL')
    #db = os.environ.get('DB_NAME')

    username = "iyappan"
    pwd = "priya159"
    host = "localhost"
    db = "test"
    
    return username,pwd,host,db

### count 

In [ ]:
query_flag = 'Q' # if query fails 
db_flag = 'D' # if connection fails
insert_success_flag = True
update_sucess_flag = True

In [ ]:
def count(tablename,condition = None):
    #defining flags 
    query_flag = 'Q'
    db_flag = 'D'
    insert_success_flag = True
    update_sucess_flag = True
    """
    Function to get the tablename and sql condition to return the count of rows which satisifies the condition in the tablename

    :param tablename: String : Table name in DB
    :param where: String : where condition to filter the table
    :return: Count of the rows for the given condition and  tablename
    """


    #connecting to the DB
    try:
        username,pwd,host,db = get_db_detail()
        connection = connect_(username,pwd,host,db)
        cursor = connection.cursor()

    except Exception as e:

        return{db_flag:str(sys.exc_info()[1])}


    if condition == None:

        query = "select count(*) from " + tablename

    else:
        #building the query
        query= "select count(*) from " + tablename + " where " + condition
        print(query)

    try:
        cursor.execute(query)
        count = cursor.fetchone()[0]
        connection.close()

        return count

    except Exception as  e:

        #closing the connection and cursor

        connection.close()

        return{query_flag:str(sys.exc_info()[1])}


### fetch one 

In [ ]:
 def fetch_one(columns,tablename,condition=None):
    #defining flags 
    query_flag = 'Q'
    db_flag = 'D'
    insert_success_flag = True
    update_sucess_flag = True
    """
    Function to return the first row of the table and where the condition satisfies

    :param columns : tuple : Columns in the table you want to view
    :param tablename: String : Tablename in the DB
    :param condition: String : Where condition to filter in the Table
    :return:  json with column schema 
    """
    # connecting to the DB
   # print(len(columns))

    if(len(columns)>1):
        cols = ""
        for i in columns:
            cols = cols + i + ","

        cols = cols[:len(cols) - 1]
    else:
        cols =str(columns)[1:len(columns)-1]


    try:
        username,pwd,host,db = get_db_detail()
        connection = connect_(username, pwd, host, db)
        cursor = connection.cursor()

    except Exception as e:

        return {db_flag: str(sys.exc_info()[1])}

    if condition == None:


        query = "select " + cols + " from " + tablename 

    else:

        query = "select " + cols + " from " + tablename + " where " + condition 



    try:
        #print(query)
        return  json.loads(pd.read_sql(query,connection).to_json(orient='records'))[0]

    except Exception as  e:

        #closing the connection and cursor

        connection.close()

        return {query_flag: str(sys.exc_info()[1])}


### fetch many

In [ ]:
def fetch_many(columns,tablename,condition=None,rows=-1):
    #defining flags 
    query_flag = 'Q'
    db_flag = 'D'
    insert_success_flag = True
    update_sucess_flag = True
    """
    Function to fetch all the values from a given table with a given condition

    :param columns : Tuple : Columns in the table you want to view
    :param tablename: String : Tablename in the DB
    :param condition: String : Where condition to filter in the Table
    :param array_size : Integer: Array Size of the cursor : Batch size
    :param rows : Integer: Number of rows to be fetched
    :return: A list of tuples  with values in order according to the columns selected from the given tablename, condition

    """


    #connecting to the DB

    try:
        username,pwd,host,db = get_db_detail()
        connection = connect_(username, pwd, host, db)
        cursor = connection.cursor()

    except Exception as e:

        return {db_flag: str(sys.exc_info()[1])}



    #converting tuples to string
    cols = ""
    for i in columns:
        cols = cols + i + ","

    cols=cols[:len(cols) - 1]

    #2 different queries : with and without where clause
    
    
    if condition == None:

        if rows == -1:
            #fetch all 
            query = "Select " + cols + " from " + tablename 
        else:
            #fetch given records 
            query = "Select " + cols + " from " + tablename +  " LIMIT " + str(rows)


    else:
        
        if rows ==-1:
            #fetch all
            query = "Select " + cols + " from " + tablename + " where " + condition
        else:
            # fetch given records 
            query = "Select " + cols + " from " + tablename + " where " + condition  + " LIMIT " + str(rows)


    try:
          
        #fetch N records 
        cursor.execute(query)
        return  json.loads(pd.read_sql(query,connection).to_json(orient='records'))

    except Exception as  e:

        #closing the connection and cursor

        connection.close()

        return {query_flag: str(sys.exc_info()[1])}



### insert one 

In [ ]:
def insert_one(tablename,object):
    #defining flags 
    query_flag = 'Q'
    db_flag = 'D'
    insert_success_flag = True
    update_sucess_flag = True
    """


    :param tablename: String : Tablename of the Database
    :param object: Dictionary : Format : {Column:Value}
    :return: True

    """


    #connecting to the DB

    try:
        username,pwd,host,db = get_db_detail()
        connection = connect_(username, pwd, host, db)
        cursor = connection.cursor()

    except Exception as e:

        return {db_flag: str(sys.exc_info()[1])}


    #preparing the columns and values

    cols = ""
    for i in tuple(object.keys()):
        cols += str(i) + ","
    cols = "(" + cols[:len(cols) - 1] + ")"

    if len(object) == 1:
        if isinstance(list(object.values())[0], str):
            val = "('" + str(tuple(object.values())[0]) + "')"
        else:
            val = "(" + str(tuple(object.values())[0]) + ")"
    else:
        val = str(tuple(object.values()))
        print(val)

    query = "INSERT INTO " + tablename + cols + " values" + val 

    #print(query)

    try:
        print(query)
        cursor.execute(query)
        connection.commit()
        connection.close()
        
        #return int(cursor.lastrowid)

    except Exception as  e:

        #closing the connection and cursor

        connection.close()

        return {query_flag: str(sys.exc_info()[1])}


### insert many

In [ ]:
def insert_all(tablename,objects,condition=None):
    #defining flags 
    query_flag = 'Q'
    db_flag = 'D'
    insert_success_flag = True
    update_sucess_flag = True
    """
      :param tablename: String : Tablename of the DB
      :param  Objects : List of Dictionary : Format : {column:value}
      :param condition: String : Where condition to filter in the Table
      :param array_size : Integer : The number of cells the cursor of the DB  should fetch in one call
      :param bind_size : Integer : The number of bindings of the object and the query should be done at a single call
      :return: True
    """

    # connecting to the DB

    try:
        username,pwd,host,db = get_db_detail()
        connection = connect_(username, pwd, host, db)
        cursor = connection.cursor()

    except Exception as e:

        return {db_flag: str(sys.exc_info()[1])}

    # preparing the columns and placeholders

    cols = objects[0].keys()
    data = []
    for i in objects:
        data.append(tuple(i.values()))
        
    #formating the reg string
    reg_string = ""
    for i in range(len(data[0])):
        reg_string+="%s, "
    reg_string = reg_string[:len(reg_string)-2]
    
    
    if condition == None:
        query = "INSERT INTO sample (%s) VALUES(%s)" % ( ",".join(cols),reg_string)
    else:
        query = "INSERT INTO sample (%s) VALUES(%s)" % ( ",".join(cols),reg_string) +" where " + condition
    
    try:
        cursor.executemany(query,data)

        connection.commit()
        connection.close()

        return insert_success_flag

    except Exception as  e:

        # closing the connection and cursor

        connection.close()

        return {query_flag: str(sys.exc_info()[1])}

### update

In [ ]:
def update(tablename,updations,condition=None):
    #defining flags 
    query_flag = 'Q'
    db_flag = 'D'
    insert_success_flag = True
    update_sucess_flag = True
    """
    :param tablename: String : Tablename of the DB
    :param updations: Object : Dictionary : Format : {column:value}
    :param condition: String : Where condition to filter in the Table
    :return: True

    """

      
    try:
        username,pwd,host,db = get_db_detail()
        connection = connect_(username, pwd, host, db)
        cursor = connection.cursor()
    except Exception as e:

        return {db_flag: str(sys.exc_info()[1])}

        # preparing the columns and values

    cols = ""

    for i in updations.keys():
        substr = str(i) + " = " + str(updations[i]) + ","
        cols = cols + substr

    cols = cols[:len(cols) - 1]

     # query creation

    if condition == None:

        query = "Update " + tablename + " set " + cols

    else:
        query = "Update " + tablename + " set " + cols + " where " + condition

    try:
        print(query)
        cursor.execute(query)

        connection.commit()
        connection.close()

        return update_sucess_flag

    except Exception as  e:

      # closing the connection and cursor

        connection.close()

        return {query_flag: str(sys.exc_info()[1])}